In [79]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 300) 
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [91]:
#purpose: create our dataframe

#default column headers
col_headers = ['Name', 'Department', 'Title', 'Regular', 'Retro', 'Other', 'Overtime', 'Injured', 'Detail', 'Quinn', 'Total', 'Postal']

#create an empty df with default values
earnings_df = pd.DataFrame(columns = col_headers)

#add column 'Year' and set to NaN
earnings_df['Year'] = np.nan

for year in range(2011, 2020):
    
    #read in the CSV for the given year, set it to variable next_df
    next_df = pd.read_csv('./data/salaries_' + str(year) + '.csv', skiprows=1, names=col_headers, encoding='latin1')
    
    #add the column 'Year' to next_df and set to the given year
    next_df['Year'] = year
    
    #in the 2013 and 2014 datasets, title and department are in the wrong order
    if year == 2013 or year == 2014:
        
        #get the list of columns (including year)
        col_list = list(next_df)
        
        #swap the order of title and department in the list
        col_list[1], col_list[2] = col_list[2], col_list[1]
        
        #set the dataframe's columns to the new list
        next_df.columns = col_list
    
    #add next_df to earnings_df
    earnings_df = pd.concat([earnings_df, next_df], sort=False)

In [92]:
#purpose: clean our dataframe

#dimensions are qualitative columns, facts are quantitative columns
dimensions = ['Name', 'Department', 'Title', 'Postal']
facts = ['Regular', 'Retro', 'Other', 'Overtime', 'Injured', 'Detail', 'Quinn', 'Total']

#targeted (hard-coded) cleaning for specific rows
earnings_df = earnings_df.loc[earnings_df['Department']!='DEPARTMENT_NAME']
earnings_df.loc[earnings_df['Department'] == 'Boston Cntr-Youth & Families', 'Department'] = 'Boston Cntr - Youth & Families'
earnings_df.loc[earnings_df['Department'] == 'DND Neighborhood Development', 'Department'] = 'Neighborhood Development'
    
#convert dimensions from type 'object' to 'string'
earnings_df[dimensions] = earnings_df[dimensions].astype(str).applymap(lambda x: x.strip())

#convert all postal codes to same format
mask = earnings_df['Postal'].str.len() == 4
earnings_df.loc[mask, 'Postal'] = '0' + earnings_df.loc[mask, 'Postal']
mask = earnings_df['Postal'].str.len() > 5 
earnings_df.loc[mask, 'Postal'] = earnings_df.loc[mask, 'Postal'].str.slice(0,5)

#clean the facts columns and convert from type 'object' to 'float'
earnings_df[facts] = earnings_df[facts].astype(str).applymap(lambda x: x.strip())
earnings_df[facts] = earnings_df[facts].replace({'^-$|^None$|^nan$|\)':0, ',':'', '\$':'', ' ':'', '^\(':'-'}, regex=True)
earnings_df[facts] = earnings_df[facts].astype(float)

#cast year to type int
earnings_df['Year'] = earnings_df['Year'].astype(int)

In [94]:
earnings_df.head(50)

Name                      Department  \
0           Abadi,Kidani A            Assessing Department   
1           Ablon,Jordan N    ASD Office Of Labor Relation   
2         Accardi,Patricia    Transportation-Parking Clerk   
3           Ackerly,Lyn E.           Boston Public Library   
4           Adams,Carey L.                  Law Department   
5               Adams,Dean         Public Works Department   
6            Adams,Natasha  Boston Cntr - Youth & Families   
7         Adario,Anthony J             ASD Human Resources   
8            Addessa,Rocco             Property Management   
9            Afonseca,Jose  Boston Cntr - Youth & Families   
10            Aguilar,Luis            Assessing Department   
11       Ahern Jr.,Fred V.  Boston Cntr - Youth & Families   
12             Ahern,Susan      Inspectional Services Dept   
13       Ahmadifar,Morteza        Neighborhood Development   
14               Akin,Gary         Public Works Department   
15          Alberque,James  Dpt of Innovation & Technology   
16      Aldrich,Kathryn A.  Comm. for Persons with Disabil   
17           Aleman,Adrian       Transportation Department   
18           Aleman,Wilson       Transportation Department   
19          Alex,George S.    Consumer Affairs & Licensing   
20     Alexander,Valerie D       Transportation Department   
21          Alexis,Jeffery         Public Works Department   
22              Alexis,Roy            Assessing Department   
23         Alfieri,Louis A         Public Works Department   
24  Alfonso-Colon,Lirida C    Transportation-Parking Clerk   
25        Ali,Omar Shairff  Dpt of Innovation & Technology   
26           Alicea,Hector                Parks Department   
27          Alkins,Jeffrey        Neighborhood Development   
28          Alkins,Jeffrey             Boston City Council   
29        Alland,Alexandra                  Law Department   
30       Allen,Alexander J         Public Works Department   
31        Allen,Archie Lee       Transportation Department   
32           Allen,Glenroy               Cemetery Division   
33           Allen,Klare X             Boston City Council   
34           Allen,R Scott                Parks Department   
35         Allen,Sharron A        Neighborhood Development   
36            Allen,Shiaka  Boston Cntr - Youth & Families   
37           Allen,William  Boston Cntr - Youth & Families   
38         Alleyne,Deborah                  Law Department   
39       Allix,Helen Marie  Boston Cntr - Youth & Families   
40      Alther,Scott James         Public Works Department   
41        Alvarez,Indira C      Inspectional Services Dept   
42        Alves,Manuela A.        Neighborhood Development   
43           Alves,Michael         Public Works Department   
44  Alves-Thomas,Andrea C.                  Law Department   
45     Amar,Erin Elizabeth        Neighborhood Development   
46        Amaral,Deborah J       Transportation Department   
47           Amato,Diane P         Public Works Department   
48           Amato,Michele            Assessing Department   
49     Ambrosino,Teresa D.    Transportation-Parking Clerk   

                             Title  Regular   Retro   Other  Overtime  \
0           Property Officer (Asn) 33065.38    0.00    0.00    379.49   
1            Asst Corp Counsel III 76051.24    0.00 1321.03      0.00   
2        Chief Claims Investigator 56430.79    0.00    0.00    418.58   
3              Spec Library Asst I 35058.78    0.00    0.00    439.19   
4                       Prin Clerk 41588.83    0.00    0.00      0.00   
5      Highway Maint Frprs (Pwd)## 37693.81    0.00 6761.54  11485.32   
6                     Youth Worker 38330.84    0.00    0.00     47.14   
7     Supvising Claims Agent (Asd) 85214.42    0.00 1629.36      0.00   
8            Jr Building Custodian 39322.79    0.00    0.00   3720.81   
9     Certified Seasonal Lifeguard  4400.45    0.00    0.00    137.99   
10      Asst Assessor (Trainee II)  1355.10    0.00    0

In [6]:
#purpose: find out how the number of employees in each department changed throughout the years

#create pivot table displaying the number of people in each dept by year
BPS_str = '^BPS |Elementary|Academy|K-8|Middle|High|School|Acad$|Pilot| EEC$| ELC$| EES$|9-12|Achievement Gap|Student|Superintendent'

def count_by_dept(column, dept=r'(.*?)'):
    #count employees where given column is greater than 0, grouped by department, rename calculated column to count
    dept_counts = earnings_df.loc[earnings_df[column]>0].groupby(['Department', 'Year'])[column].count().reset_index(name="count")
    
    #remove BPS schools from result
    dept_counts = dept_counts.loc[dept_counts['Department'].str.contains(BPS_str, regex=True) == False]
    
    #create the pivot table table, with calculated sums for each row and column
    dept_counts_table = pd.pivot_table(dept_counts, values='count', index='Department', columns='Year', aggfunc='sum', fill_value=0, margins=True).reset_index()
    
    #get rid of the calculated sums by row, doesn't make sense for time series data
    return dept_counts_table.iloc[:,0:-1].loc[dept_counts_table['Department'].str.match(dept)]

count_by_dept('Injured')

Year                      Department  2011  2012  2013  2014  2015  2016  \
0            ASD Purchasing Division     0     0     0     0     1     1   
1                         Age Strong     0     0     0     0     0     0   
2               Assessing Department     0     0     0     1     0     1   
3                Auditing Department     0     0     0     1     1     0   
4                Boston City Council     0     0     0     1     0     0   
5     Boston Cntr - Youth & Families     2     6     7     2     2     1   
6             Boston Fire Department   514   466   509  1013   495   495   
7           Boston Police Department   360   327   368   826   342   449   
8              Boston Public Library     5     1     2     2     1     1   
9                  Cemetery Division     3     3     2     4     4     2   
10    Comm. for Persons with Disabil     1     0     0     0     0     0   
11      Consumer Affairs & Licensing     0     0     0     0     0     0   
12    Dpt of Innovation & Technology     0     0     0     1     0     1   
13                Elderly Commission     3     2     1     1     1     1   
14            English Language Learn     0     0     0     0     0     0   
15             Fair Housing & Equity     0     0     0     0     0     0   
16              Food & Nutrition Svc     0     0     0     0     0     1   
17    HPEC: Com Acd Science & Health     0     0     0     0     1     1   
18        Inspectional Services Dept     9     8     6     1     5     6   
19                   Licensing Board     0     1     1     0     0     0   
20          Neighborhood Development     1     0     1     2     1     1   
21    Office of Economic Development     0     0     0     0     1     0   
22                 Office of Streets     0     0     0     0     0     0   
23                 Office of Tourism     0     0     0     0     0     1   
24                  Parks Department    14    12     4    10     7     8   
25               Property Management     4    10     5     3     3     4   
26      Public Facilities Department     0     0     0     0     0     1   
27           Public Works Department    35    39    23    30    41    34   
28                 Registry Division     0     0     0     0     1     0   
29      State Boston Retirement Syst     0     0     1     0     0     0   
30               Teaching & Learning     0     0     0     0     0     1   
31                  Traffic Division     0     0     0    33    50    40   
32         Transportation Department    64    57    40     0     0     0   
33      Transportation-Parking Clerk     0     0     1     0     1     0   
34      Workers Compensation Service     0     0    85   169   170   150   
35                               All  1015   932  1056  2100  1128  1200   

Year  2017  2018  2019  
0        0     0     1  
1        0     0     3  
2        1     0     0  
3        1     1     0  
4        0     0     0  
5        2     1     4  
6      441   442   754  
7      475   459   457  
8        2     2     3  
9        1     3     2  
10       0     0     0  
11       1     1     0  
12       1     1     0  
13       1     2     0  
14       1     0     1  
15       0     0     1  
16       0     0     0  
17       0     0     0  
18       6     5     0  
19       0     0     0  
20       1     1     0  
21       1     2     0  
22       0     0     1  
23       0     0     0  
24       9    11    10  
25       4     6     4  
26       0     0     1  
27      24    29    17  
28       0     0     0  
29       0     0     0  
30       0     0     0  
31      27    44    42  
32       0     0     0  
33       0     0     0  
34     129   110   118  
35    1128  1120  1419

In [7]:
#Arts & Cultural Development -> Office of Arts & Culture
#Transportation Department -> Traffic Division
#Women's Commission -> Women's Advancement
#Dept of Voter Mobilization -> Election Division
#Youth Fund -> Youth Engagement & Employment

In [8]:
#purpose: easily get pivot table of aggregates by department

def agg_by_dept(column, aggfunc, dept=r'(.*?)'):
    dept_costs = earnings_df.loc[earnings_df[column]>0].groupby(['Department', 'Year'])[column].agg(aggfunc).reset_index(name="Costs")
    dept_costs = dept_costs.loc[dept_costs['Department'].str.contains(BPS_str, regex=True) == False]
    dept_costs_table = pd.pivot_table(dept_costs, values='Costs', index='Department', columns='Year', aggfunc = aggfunc, fill_value=0, margins=True).reset_index()
    return dept_costs_table.iloc[:,0:-1].loc[dept_costs_table['Department'].str.match(dept)]
agg_by_dept('Injured', 'median', 'Boston Police Department|Boston Fire Department')

Year                Department     2011     2012     2013    2014     2015  \
6       Boston Fire Department 10077.10 11000.65 10033.58 2111.12  9761.53   
7     Boston Police Department  7455.53  6787.27  9174.69 1041.95 12528.32   

Year     2016    2017     2018     2019  
6     9122.28 8778.47 10325.38  6758.73  
7    10279.39 8690.22 16429.05 26432.22

In [56]:
def count_by_title(column, dept=r'(.*?)', title=r'(.*?)'):
    job_counts = earnings_df.loc[earnings_df['Department'].str.match(dept)]
    job_counts = job_counts.loc[job_counts[column]>0].groupby(['Title', 'Year'])[column].count().reset_index(name="count")
    job_counts_table = pd.pivot_table(job_counts, values='count', index='Title', columns='Year', aggfunc=np.sum, fill_value=0, margins=True).reset_index()
    return job_counts_table.loc[job_counts_table['Title'].str.match(title)].sort_values(by='All', ascending=False).iloc[:,0:-1]
count_by_title('Injured')

ValueError: Cannot index with multidimensional key

In [59]:
def agg_by_title(column, aggfunc, dept=r'(.*?)', title=r'(.*?)'):
    job_pay = earnings_df.loc[earnings_df['Department'].str.match(dept)].groupby(['Title', 'Year'])[column].agg(aggfunc).reset_index(name="Costs")
    job_pay_table = pd.pivot_table(job_pay, values='Costs', index='Title', columns='Year', fill_value=0, margins=True).reset_index()
    return job_pay_table.loc[job_pay_table['Title'].str.match(title)].sort_values(by='All', ascending=False).iloc[:,0:-1]
agg_by_title('Total', 'mean', 'Mayor\'s Office', 'Chief Diversity Officer')

Year                    Title  2011  2012  2013  2014     2015     2016  \
4     Chief Diversity Officer  0.00  0.00  0.00  0.00 97292.38 96753.68   

Year      2017      2018      2019  
4    103961.62 114576.95 112576.95

In [12]:
print(earnings_df.loc[(earnings_df['Injured']>0) & (earnings_df['Year']==2014) & (earnings_df['Department']=='Boston Police Department'), ['Name', 'Title', 'Injured']])

                     Name                           Title  Injured
17221    Abasciano,Joseph                  Police Officer 60753.78
17231       Acosta,Carina                  Police Officer  2326.64
17245    Ajemian,Gerald F  Police Offc Comm Serv Offc 3$8   999.10
17251      Alfonso,Jose M      Police Offc Acad Instr 2$6    26.25
17258       Almeida,Ana c  Police Offc/Juvenile Offc 4$10  1268.79
...                   ...                             ...      ...
20382       Younger,Atiya                  Police Officer   898.48
20383   Younger,Vatchel S                  Police Officer 11445.98
20390    Zanoli,Joseph M.                  Police Officer  2814.97
20396  Zographos,Peter A.     Police Offc Mobile Offc 2$6  1222.84
20397   Zubrin,William W.    Police Offc/Auto Invest 4$10    33.76

[826 rows x 3 columns]


In [50]:
agg_by_title('Regular', 'mean')

Year                           Title      2011      2012      2013      2014  \
849   FCommissioner/Chief of theDept      0.00      0.00      0.00      0.00   
392        Chief of Support Services      0.00      0.00      0.00 170007.27   
448               Commissioner (Bpd) 174200.00 174200.00 155194.45 186203.87   
1900                  Superintendent 266750.10 266750.12 197128.97 254357.58   
732   Dist Fire Chief-Adm Asst Dvmtr      0.00      0.00      0.00      0.00   
...                              ...       ...       ...       ...       ...   
423            Cluster Administrator      0.00      0.00      0.00      0.00   
927                 Fire Captain-ADR     -1.20      0.00      0.00      0.00   
883   FF LEP Title 3 Inspector - ADR      0.00      0.00  -1356.08   1195.32   
869       FF (Inct Comnd Sp) DEP-ADR   -569.42      0.00      0.00      0.00   
891       FF(FPD Pl of Asm Insp)-ADR  -2995.82 -14543.22      0.00      0.00   

Year      2015      2016      2017      2018      2019  
849  212438.54 201307.77 205207.77 233461.57 249615.33  
392  204892.59 207578.28 217822.94 214984.43 236325.88  
448  207017.37 221323.02 229999.90 185336.14 249999.88  
1900 126523.14 132025.96 264660.69 119230.71 132211.51  
732       0.00 174984.32 179271.73 165650.95 197961.99  
...        ...       ...       ...       ...       ...  
423       0.00      0.00      0.00      0.00      0.00  
927       0.00      0.00      0.00      0.00      0.00  
883       0.00      0.00      0.00      0.00      0.00  
869       0.00      0.00      0.00      0.00      0.00  
891       0.00      0.00      0.00      0.00      0.00  

[2065 rows x 10 columns]

In [55]:
print(earnings_df.loc[earnings_df['Title']=='Advisor'])

             Name      Department    Title   Regular  Retro     Other  \
5381  Chang,Tommy  Superintendent  Advisor 149117.42   0.00 301465.78   

      Overtime  Injured  Detail  Quinn     Total Postal  Year  
5381      0.00     0.00    0.00   0.00 450583.20  92861  2018  
